In [8]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
# suppress warnings 
import warnings
warnings.simplefilter("ignore")

In [10]:
#Importar data
FR_df=pd.read_csv("FRvideos_cc50_202101.csv")

In [11]:
#Importar el json y extraer la data que nos interesa
import json
FR_category={}
with open("FR_category_id.json") as f:
    FR_data=json.load(f)
for i in list(pd.DataFrame(FR_data['items']).id.values):
    FR_category[i]=pd.DataFrame(FR_data['items'])[pd.DataFrame(FR_data['items']).id==i].snippet.values[0]['title']

In [15]:
#vemos la informacion de toda la informacion de las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [17]:
FR_df.sample(5)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,state,lat,lon,geometry
18301,enM9WlJ2uSI,18.15.02,la petite Maëlys a été retrouvé,ٍVideo World,22,2018-02-14T19:30:04.000Z,affaire Maelys,23817,150,29,139,https://i.ytimg.com/vi/enM9WlJ2uSI/default.jpg,False,False,False,"L'unique suspect dans l'enquête sur la disparition de la petite Maëlys est passé aux aveux. Les restes du corps de la fillette ont été découverts ce mercredi. Le principal suspect, Nordahl Lelandais, a reconnu l'avoir tuée involontairement, selon les termes du procureur.\nRenaissance par Audionautix est distribué sous la licence Creative Commons Attribution (https://creativecommons.org/licenses/by/4.0/)\nArtiste : __url_artiste__",Rhone Alpes,45.899975,6.116670,POINT (6.116670287 45.89997479)
30779,TPOPB3oCG9w,18.25.04,Polymer Challenge 3 colors ! #2,MissCréatives,24,2018-04-24T13:00:04.000Z,"tuto""|""tutoriel""|""fimo""|""creation""|""misscreatives""|""miss""|""creatives""|""gâteau""|""bonbon""|""commande""|""passion""|""loisirs""|""créatifs""|""polymère""|""clay""|""cernit""|""premo""|""craft""|""area""|""scrap""|""challenge""|""polymer",7118,674,5,192,https://i.ytimg.com/vi/TPOPB3oCG9w/default.jpg,False,False,False,"∆∆ REGARDE EN BAS POUR LES INFOS ! ∆∆ \n\n\n\n• Ma chaîne de Vlog : https://www.youtube.com/channel/UCInpl_DwJVXP13OO2sXIEWg/featured\n\n\n• Facebook créations : \nhttps://www.facebook.com/MissCreativesOfficial\n\n\n\n• Mail professionnel : contact@misscreatives.fr\n\n--------------------------\n• Instagram : maevaa97\n\nLogiciel de montage : FinalCut Pro X \nCaméra : Canon EOS 650D\n\n--------------------------\n• Les questions qu'on me pose souvent ! \n\n \nMaeva, 20ans, Paris, formation caméraman !\n\n\n#Youtubeuse passionnée ! \n\n--------------------------\nN'oubli pas de me dire quelle(s) vidéo(s) t'intéresserai(ent) dans ton commentaire ! :)\n\n• #Ma.\n\n\nCertains produits présentés peuvent être des produits reçus gratuitement.",Pays De La Loire,47.480008,-0.530030,POINT (-0.530029949 47.48000755)
4966,ScDsvgGECYY,17.08.12,C'EST TOI QUI A TAPÉ MA SŒUR ?? (CAMERA CACHÉE),RAZMO,24,2017-12-06T14:30:00.000Z,"BATFLUNCH""|""CAMERA CACHEE""|""HUMOUR""|""DIVERTISSEMENT""|""ADOLESCENT""|""DROLE""|""BLAGUE""|""RAZMO""|""RASMO""|""LASALLE""|""AIDEN""|""UNWIN",208096,18074,1038,1605,https://i.ytimg.com/vi/ScDsvgGECYY/default.jpg,False,False,False,"GO 500K je compte sur toi ; Abonne toi : https://goo.gl/Nav6dS\n\nMon Instagram : https://www.instagram.com/razm0o/\n\nEssayez la nouvelle version de dr.fone ici: https://goo.gl/zQNSXS\nRoot Gratuit ici: https://goo.gl/k1ysGU\nCopiez les données de téléphone vers téléphone: https://goo.gl/WB85w2\n\nSalut à tous c'est RAZMO on se retrouve aujourd'hui pour une nouvelle vidéo , j’espère qu'elle te plaira. En tout cas n'hésite pas à liker à partager et à t'abonner . Bonne vidéo.\n\n- Suis moi également sur mes réseaux sociaux :\nSnapchat: Razmo_snap\nTwitter : https://goo.gl/7xBTy3\nBatflunch",Champagne Ardenne,48.340394,4.083358,POINT (4.083357705 48.34039431)
10477,yS9kfPIH4Rk,18.05.01,Cyril Hanouna fond en larmes devant ses chroniqueurs,Touche pas à mon poste !,24,2018-01-03T10:22:13.000Z,"TPMP""|""Cyril Hanouna""|""Touche pas à mon poste""|""C8""|""Camille Combal""|""Jean-Luc Lemoine""|""Tom Villa""|""Greg Guillotin""|""Hakim Jemili""|""Matthieu Delormeau""|""Gilles Verdez""|""Valérie Benaim""|""Agathe Auproux""|""Isabelle Morini Bosc""|""Kelly Vedovelli""|""Géraldine Maillet""|""Maxime Guény",125421,2775,278,292,https://i.ytimg.com/vi/yS9kfPIH4Rk/default.jpg,False,False,False,Tous les extraits et émissions de Touche pas à mon poste sont à retrouver sur MyCANAL : https://www.mycanal.fr/c8/tpmp/touche-pas-a-mon-poste\n\nTPMP sur les réseaux sociaux : \nFacebook : https://www.facebook.com/TPMPTV\nTwitter : https://twitter.com/TPMP\nInstagram : https://instagram.com/tp